**サポートベクトルマシン（SVM）**

* 線型分割可能（lineary separable)：直線で分割できる
* マージンの大きい分類：クラスの間にできる限り太い道を通す分類
* サポートベクトル（support vector)：道の際にあるインスタンス

SVMはスケールの影響を受けやすい！

* ハードマージン分類（hard margin classification) すべてのインスタンスが道に引っかからず、正しい側にいることを厳密に要求する分類\
    ➡線型分割できるときにしか使えない\
    ➡外れ値に敏感になりすぎる
* ソフトマージン分類（soft margin classification) 道をできる限り太くしつつ、マージン違反（margin violation)を減らす

Cが小さい→マージン違反が多くなる、汎化性能は上がることもある\
SVMモデルが過学習している場合には、Cを小さくして正則化する


In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X=iris["data"][:,(2,3)]
y=(iris["target"]==2).astype(np.float64)

svm_clf=Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

svm_clf.fit(X,y)

In [ ]:
svm_clf.predict([[5.5, 1.7]])#SVM分類器は各クラスの確率を出力しない

In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
X,y=make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf=Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X,y)


In [ ]:
y_predict = polynomial_svm_clf.predict(X)
X0 = X[(y_predict==0)]
X1 = X[(y_predict==1)]
plt.scatter(X0[:,0],X0[:,1], label="X0")
plt.scatter(X1[:,0],X1[:,1], label="X0")
plt.plot()
plt.show()

In [ ]:
import numpy as np

def plot_dataset(X, y, axes):
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$", fontsize=20)
    plt.ylabel(r"$x_2$", fontsize=20, rotation=0)

plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.show()
def plot_predictions(clf, axes):
    x0s = np.linspace(axes[0], axes[1], 100)
    x1s = np.linspace(axes[2], axes[3], 100)
    x0, x1 = np.meshgrid(x0s, x1s)
    X = np.c_[x0.ravel(), x1.ravel()]
    y_pred = clf.predict(X).reshape(x0.shape)
    y_decision = clf.decision_function(X).reshape(x0.shape)
    plt.contourf(x0, x1, y_pred, cmap=plt.cm.brg, alpha=0.2)
    plt.contourf(x0, x1, y_decision, cmap=plt.cm.brg, alpha=0.1)

plot_predictions(polynomial_svm_clf, [-1.5, 2.5, -1, 1.5])
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.show()

In [ ]:
from sklearn.svm import SVC
poly_kernel_svm_clf=Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))#degreeが多項式の次数、coefが高次多項式と低次多項式からどの程度の影響を認めるか
])

poly_kernel_svm_clf.fit(X,y)

In [ ]:
poly100_kernel_svm_clf = Pipeline([
    ("scaler",StandardScaler()),
    ("svm_clf", SVC(kernel="poly",degree=10, coef0=100, C=5))
])
poly100_kernel_svm_clf.fit(X,y)

In [ ]:
plt.figure(figsize=(11,4))

plt.subplot(121)
plot_predictions(poly_kernel_svm_clf, [-1.5,2.5,-1,1.5])
plot_dataset(X,y,[-1.5,2.5,-1,1.5])
plt.title(r"$d=3, r=1, C=5$", fontsize=18)

plt.subplot(122)
plot_predictions(poly100_kernel_svm_clf,[-1.5,2.5,-1,1.5])
plot_dataset(X,y,[-1.5,2.5,-1,1.5])
plt.title(r"$d=10, r=100, C=5$", fontsize=18)

plt.show()

* 類似性関数（similarity function):個々のインスタンスが特定のランドマーク（landmark)にどの程度近いかを測定する関数
* ガウス放射既定関数（RBF:radial basis function)：$\phi_\gamma(x,l) = \exp(-\gamma||x-l||^2)$

In [ ]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X,y)

In [ ]:
from sklearn.svm import SVC

gamma1, gamma2 = 0.1, 5
C1, C2 = 0.001, 1000
hyperparams = (gamma1, C1), (gamma1, C2), (gamma2, C1), (gamma2, C2)

svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_svm_clf=Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
    ])
    rbf_kernel_svm_clf.fit(X,y)
    svm_clfs.append(rbf_kernel_svm_clf)

plt.figure(figsize=(11,7))

for i, svm_clf in enumerate(svm_clfs):
    plt.subplot(221+i)
    plot_predictions(svm_clf, [-1.5, 2.5, -1, 1.5])
    plot_dataset(X,y,[-1.5, 2.5, -1, 1.5])
    gamm, C = hyperparams[i]
    plt.title(r"$\gamma = {}, C= $".format(gamma, C), fontsize=16)

plt.show()

カーネルの選び方\
訓練セットが大きい場合や特徴量がたくさんあるとき➡線形カーネル\
それほど大きくないとき➡ガウスRBFカーネルも試してもよい\
さらに余裕があるとき➡交差検証とグリッドサーチを使って他のカーネルを試す

SVMの考え方
* 分類→マージン違反を減らしながら2つのクラスの間に最も太い道を通す
* 回帰→マージン違反を減らしながら道の中に入るインスタンスができる限り多くなるようにする
道の太さはハイパーパラメータεによって調整\
マージンに入る訓練インスタンスを増やしても、モデルの予測に影響はない→ε不感(ε-insensitive)

In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X,y)

In [ ]:
from sklearn.svm import SVR

svm_poly_reg=SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X,y)